# Features names out

Having the ability to identify columns after applying transformations using different sklearn transformers is highly crucial. By default, sklearn transformers generate `numpy.array` outputs, lacking the column structure found in `pandas.DataFrame`.

Fortunately, a solution to this problem has been introduced in `sklearn==1.1`. This solution involves the utilization of the `get_feature_names_out` method, which returns the feature names as values.

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

from IPython.display import HTML
header_template = "<p style='font-size:17px'>{}</p>"

from sklearn.preprocessing import (
    FunctionTransformer,
    StandardScaler,
    PolynomialFeatures
)
from sklearn.datasets import make_regression

# this is frame with few numeric columns that
# is typically used for examples
input_frame = pd.DataFrame({
    f"feature{n}" : np.random.normal(0, 10, 5)
    for n in range(3)
})

## Defined names

There are converters that specify columns output names by themselves. In them you can simply call `get_features_names_out` in the trained object and the input names of the features will be returned. Some examples of such transformers:]

- `StandartScaler`;
- `OneHotEncoder`;
- `ColumnTransformer`;
- `PolynomialFeatures`;
- `CountVectorizer`;
- `TfidfVectorizer`.

### `StandartScaler`

So in the following example cell is for `StandardScaler` - it just keeps names of the input array:

In [3]:
display(HTML(header_template.format("Input dataframe")))
display(input_frame)

my_scaler = StandardScaler()
display(HTML(header_template.format("transform result")))
display(my_scaler.fit_transform(input_frame))

display(HTML(header_template.format(".get_features_names_out result")))
display(my_scaler.get_feature_names_out())

,feature0,feature1,feature2
0,-2.281355,-15.869588,10.748330
1,-3.514819,18.461833,-0.680101
2,3.049854,4.532781,11.005679
3,5.701963,-26.556241,-0.785871
4,-3.582634,0.476368,-3.538216


array([[-0.56763546, -0.76488851,  1.18730962],
       [-0.89239069,  1.40917719, -0.6467557 ],
       [ 0.83600285,  0.52710838,  1.22860973],
       [ 1.5342689 , -1.44162973, -0.66372999],
       [-0.91024559,  0.27023266, -1.10543366]])

array(['feature0', 'feature1', 'feature2'], dtype=object)

### `PolynomialFeatures`

The example we considered before was for `StandardScaler` - it returns the same amount of features as it was in the input. But what if we consider transformers that output more columns than there were in the input?

Obviously `PolynomialFeatures` belongs to such transformers in that it returns products and powers of the original features. So the next cell prints out `get_feature_names` from `PolynomialFeatures`.

In [18]:
poly_transformer.fit_transform(input_frame)

display(HTML(header_template.format("Input frame")))
display(input_frame)
display(HTML(header_template.format("Output feature names")))
display(poly_transformer.get_feature_names_out())

,feature0,feature1,feature2
0,25.020060,13.807573,-2.511767
1,-1.186492,15.497237,5.829919
2,-15.329977,10.992929,-18.756593
3,-3.080933,0.257842,-0.250496
4,2.530573,7.487708,-15.240273


array(['1', 'feature0', 'feature1', 'feature2', 'feature0^2',
       'feature0 feature1', 'feature0 feature2', 'feature1^2',
       'feature1 feature2', 'feature2^2'], dtype=object)

Everything is pretty simple - it writes the names of the inputs used in the product one by one and uses the `^` symbol to declare the power.

It seems that other transformers with such features will have something similar - quite simple, but not really customisable.

## `FunctionTransformer`

This transformer have a bit tricky behaviour of `get_feature_names_out` method. 

You have to specify `feature_names_out`. It can be `"one-to-one"` or something callable. For both options I have examples below.

### `"one-to-one"`

`one-to-one` simply causes `.get_feature_names_out` to return the feature names as they were in the input. The following example is sutitable for `dummy` transformer - the following example is suitable for the `dummy` transformer - it simply returns it's input.

In [3]:
display(HTML(header_template.format("Input dataframe")))
display(input_frame)

my_scaler = FunctionTransformer(
    lambda X : X,
    feature_names_out = "one-to-one"
)
display(HTML(header_template.format("transform result")))
display(my_scaler.fit_transform(input_frame))

display(HTML(header_template.format(".get_features_names_out result")))
display(my_scaler.get_feature_names_out())

,feature0,feature1,feature2
0,-5.263240,14.943478,-0.875921
1,-1.549951,-10.348717,9.488640
2,-0.907326,-6.827048,12.779967
3,-6.299696,-6.115877,3.503167
4,-10.672533,-16.607050,0.652093


,feature0,feature1,feature2
0,-5.263240,14.943478,-0.875921
1,-1.549951,-10.348717,9.488640
2,-0.907326,-6.827048,12.779967
3,-6.299696,-6.115877,3.503167
4,-10.672533,-16.607050,0.652093


array(['feature0', 'feature1', 'feature2'], dtype=object)

**Note**: While the usefulness of `.get_feature_names_out` may not be apparent in the current case where a transformer returns a dataframe, it becomes valuable in more complex pipelines where it remembers feature names in a scikit-learn style. Its ability to retain names can be crucial in such scenarios, making it a useful feature to have.

### Callable

If you pass callable as argument to `feature_names_out` it will be called with `.get_feature_names_out`. It should instance of `FunctionTransformer` that called method and input array of input features. 

#### Expected inputs

Let's study what is passed to the specified in `feature_names_out` funcion on the example. Let's just try to substitute such a function that will return its own input elements.

In [4]:
my_transformer = FunctionTransformer(
    lambda X: np.array(X),
    # here is just function that returns
    # it's inputs to check what they actualy
    # are
    feature_names_out=(
        lambda transformer, input_features: \
        (transformer, input_features)
    )
)
my_transformer.fit_transform(input_frame)
features_output = my_transformer.get_feature_names_out()

display(HTML(header_template.format("First argument - transformer itself")))
display(features_output[0])
print(
    "Check if first input of the feature_names_out"
    "really initial transformer - ",
    features_output[0] is my_transformer
)
display(HTML(header_template.format("Second argument - input feature names")))
features_output[1]

FunctionTransformer(feature_names_out=<function <lambda> at 0x7f1d86e37130>,
                    func=<function <lambda> at 0x7f1d86e371c0>)

Check if first input of the feature_names_outreally initial transformer -  True


array(['feature0', 'feature1', 'feature2'], dtype=object)

#### Real world example

Here is an example of how the tools we are considering might be used in the real world.

Suppose you need to build a transformer that returns squares and cubes of each feature of the input array. And the result features should have names `<inputname> square` for squares and `<inputname> cube` for cubes.

This can be achieved with the following code:

In [5]:
my_transformer = FunctionTransformer(
    # iterates over all input features
    # and returns square and cube of them
    lambda X : np.concatenate(
        [
            np.array(X[[col]]**power)
            for col in X
            for power in range(2,4)
        ], 
        axis = 1
    ),
    # iterates over input features anmes
    # and for each creates pair of square
    # and cube name
    feature_names_out = (
        lambda transformer, input_features: [
            f"{feature} {power_str}"
            for feature in input_features
            for power_str in ["square", "cubic"]
        ]
    )
)

display(HTML(header_template.format("Input dataframe")))
display(input_frame)

display(HTML(header_template.format("transform output")))
display(my_transformer.fit_transform(input_frame))

display(HTML(header_template.format(".get_feature_names_out")))
display(my_transformer.get_feature_names_out())

,feature0,feature1,feature2
0,-5.263240,14.943478,-0.875921
1,-1.549951,-10.348717,9.488640
2,-0.907326,-6.827048,12.779967
3,-6.299696,-6.115877,3.503167
4,-10.672533,-16.607050,0.652093


array([[ 2.77016967e+01, -1.45800682e+02,  2.23307526e+02,
         3.33699103e+03,  7.67237912e-01, -6.72039936e-01],
       [ 2.40234719e+00, -3.72351972e+00,  1.07095953e+02,
        -1.10830576e+03,  9.00342844e+01,  8.54302891e+02],
       [ 8.23239835e-01, -7.46946619e-01,  4.66085789e+01,
        -3.18198986e+02,  1.63327556e+02,  2.08732077e+03],
       [ 3.96861679e+01, -2.50010787e+02,  3.74039550e+01,
        -2.28757999e+02,  1.22721794e+01,  4.29914946e+01],
       [ 1.13902970e+02, -1.21563325e+03,  2.75794094e+02,
        -4.58012619e+03,  4.25224871e-01,  2.77286029e-01]])

array(['feature0 square', 'feature0 cubic', 'feature1 square',
       'feature1 cubic', 'feature2 square', 'feature2 cubic'],
      dtype=object)

#### Pandas output

Ideally, if the function specified in FunctionTransfomer returns pandas.DataFrame, then the columns that `get_feature_names_out` returns should be automatically applied to the result. But if you try to adapt the previous example, the result is an error. Just like in the following cell.

In [14]:
my_transformer = FunctionTransformer(
    # iterates over all input features
    # and returns square and cube of them
    lambda X : pd.concat(
        [
            X[col]**power
            for col in X
            for power in range(2,4)
        ],
        axis=1
    ),
    # iterates over input features anmes
    # and for each creates pair of square
    # and cube name
    feature_names_out = (
        lambda transformer, input_features: [
            f"{feature} {power_str}"
            for feature in input_features
            for power_str in ["square", "cubic"]
        ]
    )
)
my_transformer.set_output(transform='pandas')

try:
    my_transformer.fit_transform(input_frame)
except Exception as e:
    print(e)

The output generated by `func` have different column names than the ones provided by `get_feature_names_out`. Got output with columns names: ['feature0', 'feature0', 'feature1', 'feature1', 'feature2', 'feature2'] and `get_feature_names_out` returned: ['feature0 square', 'feature0 cubic', 'feature1 square', 'feature1 cubic', 'feature2 square', 'feature2 cubic']. The column names can be overridden by setting `set_output(transform='pandas')` or `set_output(transform='polars')` such that the column names are set to the names provided by `get_feature_names_out`.


I think it's a `sklearn` bug, so I've created a corresponding [issue](https://github.com/scikit-learn/scikit-learn/issues/28780). Check it out, there might be a solution already.

## Building the whole pipeline

All tools described above are useful only in case if we are using them in real life.

In [178]:
np.random.seed(100)

n_features=20
n_samples=1000

X, y = make_regression(
    n_samples=n_samples,
    n_features=n_features, 
    n_informative=10,
    random_state=10
)

def categorise_feature(X : np.array) -> np.array:
    '''
    Function that allows to convert numerical
    characteristics into categorical characteristics 
    with Latin alphabet. It uses a random number 
    of bins.

    Parameters
    ----------
    X : np.array
        input feature;

    Returns
    ----------
    out : np.array
        same size feature with values
        encoded as symbols of Latin alphabet.
    '''
    quantiles = np.linspace(0, 1, np.random.randint(4, 10))[1:-1]
    bins = np.quantile(X, quantiles)
    labels = [
        chr(i) for i in 
        range(
            ord("a"), 
            ord("a") + len(quantiles) + 1
        )
    ]
    res = np.full(len(X), labels[0])
    for bin, label in zip(bins, labels[1:]):
        res[X>bin] = label

    return res

X = pd.DataFrame({
    f"feature {i}":(
        X[:, i]*np.random.uniform(-100,100) 
        if np.random.rand() > 0.5 else 
        categorise_feature(X[:,i])
    ) 
    for i in range(X.shape[1])
})

# constructing month number which
# correlate with target
months = y + np.random.normal(0, np.std(y)/10, n_samples)
months = np.round((
    (monthes - np.min(monthes))/
    (np.max(monthes) - np.min(monthes))
)*12)
months[months < 1] = 1

# constructing date feature that using
# where only month can be useful in
# the prediction task
date_feature = (
    pd.DataFrame({
        "day" : np.random.randint(1,29, n_samples),
        "month" : months.astype("int"),
        "year" : np.random.randint(1980, 2024, n_samples)
    })
    .apply(
        lambda row: datetime(
            row["year"], 
            row["month"], 
            row["day"]
        ), 
        axis = 1
    )
)

X["date feature"] = date_feature
X.head()

,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 11,feature 12,feature 13,feature 14,feature 15,feature 16,feature 17,feature 18,feature 19,date feature
0,2.617891,c,d,b,c,a,a,b,-196.739681,e,...,-47.603852,d,33.356150,a,a,-122.367117,-6.721474,c,e,1986-08-04
1,-70.125115,a,a,c,a,a,b,b,-10.601175,f,...,-71.627628,a,-101.127295,e,b,-91.372820,4.377953,c,d,1993-09-14
2,-20.582671,c,a,c,d,c,b,b,200.739023,d,...,-16.353255,a,40.661435,a,a,-96.068509,0.313386,b,g,1991-07-07
3,-15.356126,c,a,d,c,c,d,b,-153.459105,b,...,26.679386,e,120.183909,f,b,16.846444,-2.223026,a,f,2013-06-02
4,-63.181240,a,g,a,b,b,d,b,-108.291832,b,...,-17.332633,f,21.126076,b,b,-147.533723,-2.960036,b,h,2019-09-16
